In [1]:
pip install librosa tensorflow scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
# Define the Config class
class Config:
    SR = 32000
    N_MFCC = 13
    # Dataset
    N_CLASSES = 2
    BATCH_SIZE = 128
    N_EPOCHS = 5
    LR = 1e-4
    DROPOUT_RATE = 0.3
    SEED = 42

CONFIG = Config()

# Function to set the seed for reproducibility
def seed_everything(seed):
    np.random.seed(seed)

seed_everything(CONFIG.SEED) # Fix the seed

# Feature Extraction function
def extract_features(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=CONFIG.SR)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfcc.T

# Load your DataFrame
train_df = pd.read_csv('C:/Users/KimDongyoung/Downloads/SW중심대학/train.csv')
test_df = pd.read_csv('C:/Users/KimDongyoung/Downloads/SW중심대학/test.csv')

In [9]:
train_df

,id,path,label
0,RUNQPNJF,./train/RUNQPNJF.ogg,real
1,JFAWUOGJ,./train/JFAWUOGJ.ogg,fake
2,RDKEKEVX,./train/RDKEKEVX.ogg,real
3,QYHJDOFK,./train/QYHJDOFK.ogg,real
4,RSPQNHAO,./train/RSPQNHAO.ogg,real
...,...,...,...
55433,NJGPMAJL,./train/NJGPMAJL.ogg,real
55434,SZXIACUZ,./train/SZXIACUZ.ogg,fake
55435,PXLBTGRH,./train/PXLBTGRH.ogg,fake
55436,CGGQGPOQ,./train/CGGQGPOQ.ogg,fake


In [8]:
# Assuming the 'path' column is present and contains the file paths
# Prepare the data
def prepare_data(df, root_folder):
    features = []
    labels = []
    for i, row in df.iterrows():
        file_path = os.path.join(root_folder, row['path'])
        mfcc = extract_features(file_path, CONFIG.N_MFCC)
        features.append(mfcc)
        labels.append(row['label'])
    return features, labels

# Assuming root_folder is where the audio files are located
root_folder = r"C:\Users\KimDongyoung\Downloads\SW중심대학"

train_features, train_labels = prepare_data(train_df, root_folder)
val_features, val_labels = prepare_data(test_df, root_folder)


KeyError: 'label'

In [ ]:
# Pad sequences to the same length
max_len = max(max(len(x) for x in train_features), max(len(x) for x in val_features))
train_features = pad_sequences(train_features, maxlen=max_len, dtype='float32', padding='post')
val_features = pad_sequences(val_features, maxlen=max_len, dtype='float32', padding='post')

train_features = np.array(train_features)
train_labels = np.array(train_labels)
val_features = np.array(val_features)
val_labels = np.array(val_labels)

# Custom callback to calculate F1 score
class F1ScoreCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_predict = (np.asarray(self.model.predict(val_features))).round()
        val_predict = np.argmax(val_predict, axis=1)
        f1 = f1_score(val_labels, val_predict, average='weighted')
        print(f' — val_f1_score: {f1}')

# Model Definition
model = Sequential()
model.add(LSTM(64, input_shape=(train_features.shape[1], train_features.shape[2]), return_sequences=True))
model.add(Dropout(CONFIG.DROPOUT_RATE))
model.add(LSTM(64))
model.add(Dropout(CONFIG.DROPOUT_RATE))
model.add(Dense(CONFIG.N_CLASSES, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training with F1 score callback
f1_callback = F1ScoreCallback()
history = model.fit(train_features, train_labels, epochs=CONFIG.N_EPOCHS, batch_size=CONFIG.BATCH_SIZE,
                    validation_data=(val_features, val_labels), callbacks=[f1_callback])

# Final Evaluation with F1 score
val_predict = (np.asarray(model.predict(val_features))).round()
val_predict = np.argmax(val_predict, axis=1)
f1 = f1_score(val_labels, val_predict, average='weighted')
print(f'Final Validation F1 Score: {f1}')